In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import warnings
from nrclex import NRCLex
warnings.filterwarnings('ignore')

### Introducción
El uso de lexicones en el analisis de datos en general, nos permite comenzar a clasificar y saber quer tipo de reaccion se nos esta dando en el feeback, comentarios, mensajes, etc.
Esto puede ser usado para la toma de desiciones empresariales, de igual forma nos permite conocer si la comunidad apoya alguna accion recientemente realziada por algun grupo o bien si el producto/servicio tiene buena calidad, etc.

## Lectura de datos
Para este proyecto se utilizaran datos financieros disponibles de forma gratuita en el siguiente [enlace](https://www.kaggle.com/ankurzing/sentiment-analysis-for-financial-news).

Estos vienen estructurados con dos columnas:
* Sentimiento generado
* Comentario

In [2]:
data = pd.read_csv('./archive/all-data.csv',header=None)
data = data.rename(columns={0:'feel',1:'review'})
data.head()

,feel,review
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


### Limpieza
Como podemos ver, estos datos tienen palabras que no nos serian utiles por ejemplo:
* Números
* Signos de puntuación
* Pronombres
* StopWords (palabras como 'The', 'In','out', etc.)

Por lo que necesitaremos hacer una limpieza de estos datos.

Para ello utilizando la libreria NLTK descargaremos un conjunto de datos que contiene las stopwords de diferentes idiomas

In [3]:
# Descarga de conjunto
nltk.download('stopwords')

#Seleccion de lenguaje
stop = stopwords.words('english')
stop[:10]

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/famvazpom/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

#### Función utilizada para la limpieza

#### Continuando con la limpieza
Primero que nada, copiaremos el conjunto de datos a un dataframe diferente.
Posteriormente realizamos las siguientes acciones:
* Eliminamos signos de puntuación
* Eliminamos números
* Realizamos una limpieza de multiples espacios consecutivos para tener solamente un espacio
* Eliminamos las StopWords

In [4]:
data_cleaned = data.copy()
data_cleaned['review'] = data_cleaned['review'].str.replace("[^\w\s]",'').str.lower()
data_cleaned['review'] = data_cleaned['review'].str.replace("[^\D]",'').str.lower()
data_cleaned['review'] = data_cleaned['review'].str.replace(" {2,}",' ').str.lower()
data_cleaned['review'] = data_cleaned['review'].apply(lambda x: [item for item in x.split() if item not in stop])
data_cleaned['review']

0       [according, gran, company, plans, move, produc...
1       [technopolis, plans, develop, stages, area, le...
2       [international, electronic, industry, company,...
3       [new, production, plant, company, would, incre...
4       [according, company, updated, strategy, years,...
                              ...                        
4841    [london, marketwatch, share, prices, ended, lo...
4842    [rinkuskiai, beer, sales, fell, per, cent, mil...
4843    [operating, profit, fell, eur, mn, eur, mn, in...
4844    [net, sales, paper, segment, decreased, eur, m...
4845    [sales, finland, decreased, january, sales, ou...
Name: review, Length: 4846, dtype: object

#### Lexicones
Despues de la limpieza podemos comenzar con la polarización de datos en los 'reviews'.

Para ello utilizaremos 2 lexicones diferentes que nos permitiran conocer la polaridad de esto, los lexicones utilizados son:

* Vader
* Punkt

Por lo que descargaremos mediante NLTK

In [5]:
nltk.download('vader_lexicon')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/famvazpom/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /home/famvazpom/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Función utilizada para el procesamiento
Esta función obtiene todas las palabras de cada uno de los reviews y dependiendo la puntuación obtenida, realiza la sumatoria de estos en 3 diferentes categorias:
* Positivo
* Negativo
* Neutral

Y finalmente regresa el label de la categoria con mayor puntuación.

In [6]:
def get_polarity(data,lexicon):
    pos = 0
    neg = 0
    neut = 0
    for word in data:
        value = lexicon.get(word,0)
        if value <0:
            neg += value
        elif value >1:
            pos += value
        else:
            neut += value 
    values = {abs(neg):'negative',pos:'positive',neut:'neutral'}

    return values[max(values)]


#### Utilizando la funcion anterior
Los resultados de la función anterior nos los primeros 5 resultados similares.

In [7]:
sent = SentimentIntensityAnalyzer()
lexicon =  sent.make_lex_dict()
data_cleaned['feel_student'] = data_cleaned['review'].apply(lambda review: get_polarity(review,lexicon))
data_cleaned[['feel_student','feel']].head()

,feel_student,feel
0,neutral,neutral
1,neutral,neutral
2,neutral,negative
3,positive,positive
4,positive,positive


Sin embargo, el lexicon utilizado nos permite utilizar su propio clasificador por lo que obtendremos cual es la polaridad con mayor puntuacion utilizando el clasificador proporcionado por Vader, pero para esto, necesitamos convertir el review en un solo string para funcionar por lo que utilizaremos la siguiente función:

In [8]:
def listToString(lst):
    string = ' '
    return string.join(lst)

Posteriormente, realizaremos el procesamiento para esto.

In [9]:
data_cleaned['review_str'] = data_cleaned['review'].apply(lambda x: listToString(x))

### Funcion que obtiene el sentimiento con mayor puntuación.

In [10]:
def get_max_polarity(review,sentiment):
    scores = sentiment.polarity_scores(review)
    val = {}
    for key in scores.keys():
        val[scores[key]] = key
    return val[max(val)]

Con los resultados siguientes obtenemos una nueva categoria que nos dice que es una mezcla de diferentes sentimientos.

In [11]:
data_cleaned['Vader'] = data_cleaned['review_str'].apply(lambda review: get_max_polarity(review,sent))
data_cleaned['Vader'].head()
data_cleaned[['feel','feel_student','Vader']].head()

,feel,feel_student,Vader
0,neutral,neutral,neu
1,neutral,neutral,neu
2,negative,neutral,neu
3,positive,positive,compound
4,positive,positive,neu


### Utilizando el Lexicon NRCLex
Este lexicon nos da diferentes sentimientos, por lo que obtendremos aquel que nos de una mayor puntuación.

In [12]:
def get_top_emotions(data):
    results = NRCLex(data).top_emotions
    val = {}
    for i in results:
        val[i[1]] = i[0]
    return val[max(val)]

Con los resultados del clasificador anterior, podemos ver que obtenemos diferentes valores, como tal en los primeros 5 records podemos observar 2 nuevos sentimientos tales como:
* Anticipación
* Tristesa

In [13]:
data_cleaned['NRCLex'] = data_cleaned['review_str'].apply(lambda review: get_top_emotions(review))
data_cleaned[['feel','feel_student','Vader','NRCLex']].head()

,feel,feel_student,Vader,NRCLex
0,neutral,neutral,neu,anticipation
1,neutral,neutral,neu,positive
2,negative,neutral,neu,negative
3,positive,positive,compound,positive
4,positive,positive,neu,sadness


# Vazquez Pompa Noe - 160300153